In [1]:
from langchain.llms import GooglePalm
import os
from dotenv import load_dotenv
load_dotenv() 

api_key = os.environ["GOOGLE_API_KEY"]

llm = GooglePalm(google_api_key=api_key, temperature=0.2)

In [3]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from sqlalchemy import create_engine

In [ ]:
db_driver = "ODBC Driver 17 for SQL Server"
db_server = "NAVEENTAMIZHAN"
db_name = "nav_electric_vehicles"
Database_Connect = f"mssql://@{db_server}/{db_name}?Driver={db_driver}"

engine = create_engine(Database_Connect)
db = SQLDatabase(engine, sample_rows_in_table_info=3)

print(db.table_info)

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How many tesla do we have left for tesla in premium model and white color?")